In [236]:
!pip install bs4
!pip install gdown
from bs4 import BeautifulSoup
import requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [250]:
class Yes24ScrapingService:
    url_format = 'http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}'

    def get_book_info(self, isbn : str) -> BeautifulSoup:
        url = Yes24ScrapingService.url_format.format(isbn = isbn)

        html = requests.get(url)
        html_formatted = BeautifulSoup(html.text, 'html.parser')

        book_info = html_formatted.find('a', attrs={'class':'gd_name'})
        if book_info == None:
            return 0

        book_url = 'http://www.yes24.com'+book_info['href']
        book_html = requests.get(book_url)
        book_formatted = BeautifulSoup(book_html.text, 'html.parser')
        return book_formatted


    def get_book_page_count(self, isbn : str) -> int:
        book_formatted = self.get_book_info(isbn)

        book_infoset_specific = book_formatted.find('div', attrs={'id':'infoset_specific'})

        prd_tr_list = book_infoset_specific.find_all('tr')
        for tr in prd_tr_list:
            if tr.find('th').get_text() == '쪽수, 무게, 크기':
                values = tr.find('td').get_text().split()
                page_count = int(values[0].split('쪽')[0])
                return page_count
        return 0

    def get_book_weight(self, isbn : str) -> int:
        """
        책의 무게를 g 단위로 반환합니다.
        """
        book_formatted = self.get_book_info(isbn)

        book_infoset_specific = book_formatted.find('div', attrs={'id':'infoset_specific'})

        prd_tr_list = book_infoset_specific.find_all('tr')
        for tr in prd_tr_list:
            if tr.find('th').get_text() == '쪽수, 무게, 크기':
                values = tr.find('td').get_text().split()
                weight = values[2].split('g')[0]
                return weight
        return 0

    def get_book_size(self, isbn : str) -> str:
        book_formatted = self.get_book_info(isbn)

        book_infoset_specific = book_formatted.find('div', attrs={'id':'infoset_specific'})

        prd_tr_list = book_infoset_specific.find_all('tr')
        for tr in prd_tr_list:
            if tr.find('th').get_text() == '쪽수, 무게, 크기':
                values = tr.find('td').get_text().split()
                size = values[4]
                return size
        return 0

    def get_book_author_name(self, isbn : str) -> str:
        book_formatted = self.get_book_info(isbn)
        book_lnk_author = book_formatted.find('a', attrs={'class':'lnk_author'})
        book_author_name = str(book_lnk_author).split('>')[-2].split('<')[0]
        return book_author_name

def get_page_cnt(isbn : str) -> int:
    return Yes24ScrapingService().get_book_page_count(isbn)


In [251]:
page_cnt = get_page_cnt(9791171710577)
print(page_cnt)

book_page_count = Yes24ScrapingService().get_book_page_count(9791171710577)
book_size = Yes24ScrapingService().get_book_size(9791171710577)
book_weight = Yes24ScrapingService().get_book_weight(9791171710577)
book_author_name = Yes24ScrapingService().get_book_author_name(9791171710577)

print(book_page_count)
print(book_size)
print(book_weight)
print(book_author_name)

300
300
130*195*20mm
518
이석원


In [239]:
import gdown

gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

import pandas as pd

books = pd.read_json('20s_best_book.json')

Downloading...
From: https://bit.ly/3q9SZix
To: /Users/kkimj/MOT4045/assignments/20s_best_book.json
100%|██████████| 92.9k/92.9k [00:00<00:00, 948kB/s]


In [240]:
ranking_books = books.head(5)

def get_book_weight(row):
    isbn = row['isbn13']
    return Yes24ScrapingService().get_book_weight(isbn)

ranking_books_with_weight = ranking_books.apply(get_book_weight, axis=1)
print(ranking_books_with_weight)

0    496
1    358
2    296
3    412
4    388
dtype: object


In [252]:
ranking_books_weight = ranking_books.apply(
    lambda row: Yes24ScrapingService().get_book_weight(row['isbn13']), axis=1)

ranking_books_weight.name = 'weight (gram)'
print(ranking_books_weight)

0    496
1    358
2    296
3    412
4    388
Name: weight (gram), dtype: object


In [253]:
ranking_books_with_weight = pd.merge(
    ranking_books, ranking_books_weight, left_index=True, right_index=True)
ranking_books_with_weight

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,weight (gram)
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,461,https://image.aladin.co.kr/product/19359/16/co...,496
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,387,https://image.aladin.co.kr/product/24512/70/co...,358
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/19804/82/co...,296
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,370,https://image.aladin.co.kr/product/24131/37/co...,412
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,365,http://image.aladin.co.kr/product/16839/4/cove...,388


# 2

In [254]:
def get_book_page_count(row):
    isbn = row['isbn13']
    return Yes24ScrapingService().get_book_page_count(isbn)

In [255]:
page_count = ranking_books_with_weight.apply(get_book_page_count, axis=1)
print(page_count)

0    330
1    300
2    228
3    340
4    264
dtype: int64


In [256]:
ranking_books_page_count = ranking_books.apply(
    lambda row: Yes24ScrapingService().get_book_page_count(row['isbn13']), axis=1)
print(ranking_books_page_count)

0    330
1    300
2    228
3    340
4    264
dtype: int64


In [257]:
ranking_books_page_count.name = 'page_count'
print(ranking_books_page_count)

0    330
1    300
2    228
3    340
4    264
Name: page_count, dtype: int64


In [258]:
ranking_books_with_weight_page_count = pd.merge(
    ranking_books_with_weight, ranking_books_page_count, left_index=True, right_index=True)
ranking_books_with_weight_page_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,weight (gram),page_count
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,461,https://image.aladin.co.kr/product/19359/16/co...,496,330
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,387,https://image.aladin.co.kr/product/24512/70/co...,358,300
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/19804/82/co...,296,228
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,370,https://image.aladin.co.kr/product/24131/37/co...,412,340
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,365,http://image.aladin.co.kr/product/16839/4/cove...,388,264
